In [19]:
from asyncio import protocols
from datetime import datetime,timedelta,date
import pandas as pd
from pyhive import presto
import numpy as np
import os
import warnings
from scipy import stats
import datetime
presto_host = 'bi-presto.serving.data.production.internal' #'bi-presto.serving.data.production.internal' #'presto.processing.yoda.run' #'bi-presto.serving.data.production.internal'
presto_port = '80'
presto_conn = presto.connect(
    host=presto_host,
    port=presto_port,
    username='praveen.u@rapido.bike',
    protocol='http',
    catalog='hive',
)
pd.options.mode.chained_assignment = None

In [20]:
#AOR

In [24]:
data_df_v2=pd.DataFrame()

for i in pd.date_range('20230522','20230529'):

    query="""with order_logs as ( 
                            select captain_id,yyyymmdd,hour,order_id,city_name,order_status,event_type_agg
                            from
                            (
                            select * from
                            (
                            select  yyyymmdd,
                            order_id,captain_id,city_name,order_status,substr(hhmmss,1,2) as hour,
                            array_agg(event_type) over (partition by order_id) as event_type_agg,
                            row_number() over(partition by order_id order by updated_epoch desc) as row 
                            from  orders.order_logs_immutable 
                            where yyyymmdd = '{std}'
                            and city_name in ('Hyderabad','Chennai')
                            and order_id in  (select distinct order_id 
                                                            from 
                                                            orders.order_logs_snapshot
                                                            where yyyymmdd = '{std}'
                                                            and service_obj_service_name = 'Auto'
                                                            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
                                                            and lower(trim(city_name)) in ('hyderabad','chennai'))
                            )
                            ) as a
                            )
                
                select city_name,yyyymmdd,hour,
                Accepted_Orders/(gross_orders*1.00) as AOR
                
                from
                (
                    select city_name,yyyymmdd,hour,
                    count(distinct case when contains(event_type_agg, 'accepted') then order_logs.order_id end) as Accepted_Orders,
                    count(distinct order_logs.order_id) as gross_orders

                    from order_logs
                    group by 1,2,3
                )
                """.format(std=i.strftime('%Y%m%d'))

    data1=pd.read_sql(query,presto_conn)
    print(i.strftime('%Y%m%d'))
    
    #print(len(data1))
    
    data_df_v2=data_df_v2.append(data1)


20230522
20230523
20230524
20230525
20230526
20230527
20230528
20230529


In [25]:
data_df_v2=data_df_v2.sort_values(by=['city_name','hour']).reset_index(drop=True)

In [27]:
data_df_v2.to_clipboard()